# Introduction

In [1]:
import tiledb.plot.widget

In [2]:
mock_data = """{
   "nodes":[
    "node_1",
    "node_2",
    "node_4",
    "node_6",
    "node_3",
    "node_5",
    "node_8",
    "node_7",
    "node_9",
    "node_10",
    "node_11",
    "node_12",
    "node_13",
    "node_14",
    "node_15",
    "node_16",
    "node_17",
    "node_18",
    "node_19",
    "node_20",
    "node_21",
    "node_22",
    "node_23",
    "node_24",
    "node_25",
    "node_26",
    "node_27",
    "node_28",
    "node_29",
    "node_30",
    "node_31",
    "node_32",
    "node_33",
    "node_34",
    "node_35",
    "node_36",
    "node_37",
    "node_38",
    "node_39",
    "node_40",
    "node_41",
    "node_42",
    "node_43",
    "node_44",
    "node_45",
    "node_46",
    "node_47",
    "node_48",
    "node_49",
    "node_50",
    "node_51",
    "node_52",
    "node_53",
    "node_54",
    "node_55",
    "node_56",
    "node_57",
    "node_58",
    "node_59",
    "node_60",
    "node_61",
    "node_62",
    "node_63",
    "node_64",
    "node_65",
    "node_66",
    "node_67",
    "node_68",
    "node_69",
    "node_70",
    "node_71",
    "node_72",
    "node_73",
    "node_74",
    "node_75",
    "node_76",
    "node_77",
    "node_78",
    "node_79",
    "node_80",
    "node_81",
    "node_82",
    "node_83",
    "node_84",
    "node_85",
    "node_86",
    "node_87",
    "node_88",
    "node_89"
   ],
   "edges":[
    [
        "node_1",
        "node_2"
     ],
     [
        "node_1",
        "node_3"
     ],
     [
        "node_1",
        "node_4"
     ],
     [
        "node_1",
        "node_5"
     ],
     [
        "node_1",
        "node_6"
     ],
     [
        "node_1",
        "node_7"
     ],
     [
        "node_1",
        "node_8"
     ],
     [
        "node_1",
        "node_9"
     ],
     [
        "node_1",
        "node_10"
     ],
     [
        "node_1",
        "node_11"
     ],
     [
        "node_1",
        "node_12"
     ],
     [
        "node_1",
        "node_13"
     ],
     [
        "node_1",
        "node_14"
     ],
     [
        "node_1",
        "node_15"
     ],
     [
        "node_1",
        "node_16"
     ],
     [
        "node_1",
        "node_17"
     ],
     [
        "node_1",
        "node_18"
     ],
     [
        "node_1",
        "node_19"
     ],
     [
        "node_1",
        "node_20"
     ],
     [
        "node_1",
        "node_21"
     ],
     [
        "node_1",
        "node_22"
     ],
     [
        "node_1",
        "node_23"
     ],
     [
        "node_1",
        "node_24"
     ],
     [
        "node_1",
        "node_25"
     ],
     [
        "node_1",
        "node_26"
     ],
     [
        "node_1",
        "node_27"
     ],
     [
        "node_1",
        "node_28"
     ],
     [
        "node_1",
        "node_29"
     ],
     [
        "node_1",
        "node_30"
     ],
     [
        "node_1",
        "node_31"
     ],
     [
        "node_1",
        "node_32"
     ],
     [
        "node_1",
        "node_33"
     ],
     [
        "node_1",
        "node_34"
     ],
     [
        "node_1",
        "node_35"
     ],
     [
        "node_1",
        "node_36"
     ],
     [
        "node_1",
        "node_37"
     ],
     [
        "node_1",
        "node_38"
     ],
     [
        "node_1",
        "node_39"
     ],
     [
        "node_1",
        "node_40"
     ],
     [
        "node_1",
        "node_41"
     ],
     [
        "node_1",
        "node_42"
     ],
     [
        "node_1",
        "node_43"
     ],
     [
        "node_1",
        "node_44"
     ],
     [
        "node_1",
        "node_45"
     ],
     [
        "node_1",
        "node_46"
     ],
     [
        "node_1",
        "node_47"
     ],
     [
        "node_1",
        "node_48"
     ],
     [
        "node_1",
        "node_49"
     ],
     [
        "node_1",
        "node_50"
     ],
     [
        "node_1",
        "node_51"
     ],
     [
        "node_1",
        "node_52"
     ],
     [
        "node_1",
        "node_53"
     ],
     [
        "node_1",
        "node_54"
     ],
     [
        "node_1",
        "node_55"
     ],
     [
        "node_1",
        "node_56"
     ],
     [
        "node_1",
        "node_57"
     ],
     [
        "node_1",
        "node_58"
     ],
     [
        "node_1",
        "node_59"
     ],
     [
        "node_1",
        "node_60"
     ],
     [
        "node_1",
        "node_61"
     ],
     [
        "node_1",
        "node_62"
     ],
     [
        "node_1",
        "node_63"
     ],
     [
        "node_1",
        "node_64"
     ],
     [
        "node_1",
        "node_65"
     ],
     [
        "node_1",
        "node_66"
     ],
     [
        "node_1",
        "node_67"
     ],
     [
        "node_1",
        "node_68"
     ],
     [
        "node_1",
        "node_69"
     ],
     [
        "node_1",
        "node_70"
     ],
     [
        "node_1",
        "node_71"
     ],
     [
        "node_1",
        "node_72"
     ],
     [
        "node_1",
        "node_73"
     ],
     [
        "node_1",
        "node_74"
     ],
     [
        "node_1",
        "node_75"
     ],
     [
        "node_1",
        "node_76"
     ],
     [
        "node_1",
        "node_77"
     ],
     [
        "node_1",
        "node_78"
     ],
     [
        "node_1",
        "node_79"
     ],
     [
        "node_1",
        "node_80"
     ],
     [
        "node_1",
        "node_81"
     ],
     [
        "node_1",
        "node_82"
     ],
     [
        "node_1",
        "node_83"
     ],
     [
        "node_1",
        "node_84"
     ],
     [
        "node_1",
        "node_85"
     ],
     [
        "node_1",
        "node_86"
     ],
     [
        "node_1",
        "node_87"
     ],
     [
        "node_1",
        "node_88"
     ],
     [
        "node_1",
        "node_89"
     ]
   ],
   "node_details":{
      "node_1":{
         "status":"Completed",
         "name":"node_1"
      },
      "node_2":{
         "status":"Completed",
         "name":"e85ef692-41f6-4bc7-bdfa-761569233e2f"
      },
      "node_3":{
         "status":"Failed",
         "name":"node_3"
      },
      "node_4":{
         "status":"Running",
         "name":"node_4"
      },
      "node_5":{
         "status":"Running",
         "name":"node_5"
      },
      "node_6":{
         "status":"Not Started",
         "name":"node_6"
      },
      "node_7":{
         "status":"Cancelled",
         "name":"node_7"
      },
      "node_8":{
         "status":"Not Started",
         "name":"node_8"
      },
      "node_9":{
         "status":"Not Started",
         "name":"node_9"
      },
      "node_10":{
         "status":"Running",
         "name":"node_10"
      },
      "node_11":{
         "status":"Not Started",
         "name":"node_11"
      },
      "node_12":{
        "status":"Not Started",
        "name":"node_12"
     },
     "node_13":{
        "status":"Not Started",
        "name":"node_13"
     },
     "node_14":{
        "status":"Not Started",
        "name":"node_14"
     },
     "node_15":{
        "status":"Not Started",
        "name":"node_15"
     },
     "node_16":{
        "status":"Not Started",
        "name":"node_16"
     },
     "node_17":{
        "status":"Not Started",
        "name":"node_17"
     },
     "node_18":{
        "status":"Not Started",
        "name":"node_18"
     },
     "node_19":{
        "status":"Not Started",
        "name":"node_19"
     },
     "node_20":{
        "status":"Not Started",
        "name":"node_20"
     },
     "node_21":{
        "status":"Not Started",
        "name":"node_21"
     },
     "node_22":{
       "status":"Not Started",
       "name":"node_22"
    },
    "node_23":{
       "status":"Not Started",
       "name":"node_23"
    },
    "node_24":{
       "status":"Not Started",
       "name":"node_24"
    },
    "node_25":{
       "status":"Not Started",
       "name":"node_25"
    },
    "node_26":{
       "status":"Not Started",
       "name":"node_26"
    },
    "node_27":{
       "status":"Not Started",
       "name":"node_27"
    },
    "node_28":{
       "status":"Not Started",
       "name":"node_28"
    },
    "node_29":{
       "status":"Not Started",
       "name":"node_29"
    },
    "node_30":{
        "status":"Not Started",
        "name":"node_30"
     },
     "node_31":{
        "status":"Not Started",
        "name":"node_31"
     },
     "node_32":{
       "status":"Not Started",
       "name":"node_32"
    },
    "node_33":{
       "status":"Not Started",
       "name":"node_33"
    },
    "node_34":{
       "status":"Not Started",
       "name":"node_34"
    },
    "node_35":{
       "status":"Not Started",
       "name":"node_35"
    },
    "node_36":{
       "status":"Not Started",
       "name":"node_36"
    },
    "node_37":{
       "status":"Not Started",
       "name":"node_37"
    },
    "node_38":{
       "status":"Not Started",
       "name":"node_38"
    },
    "node_39":{
       "status":"Not Started",
       "name":"node_39"
    },
    "node_40":{
        "status":"Not Started",
        "name":"node_40"
     },
     "node_41":{
        "status":"Not Started",
        "name":"node_41"
     },
     "node_42":{
       "status":"Not Started",
       "name":"node_42"
    },
    "node_43":{
       "status":"Not Started",
       "name":"node_43"
    },
    "node_44":{
       "status":"Not Started",
       "name":"node_44"
    },
    "node_45":{
       "status":"Not Started",
       "name":"node_45"
    },
    "node_46":{
       "status":"Not Started",
       "name":"node_46"
    },
    "node_47":{
       "status":"Not Started",
       "name":"node_47"
    },
    "node_48":{
       "status":"Not Started",
       "name":"node_48"
    },
    "node_49":{
       "status":"Not Started",
       "name":"node_49"
    },
    "node_50":{
        "status":"Not Started",
        "name":"node_50"
     },
     "node_51":{
        "status":"Not Started",
        "name":"node_51"
     },
     "node_52":{
       "status":"Not Started",
       "name":"node_52"
    },
    "node_53":{
       "status":"Not Started",
       "name":"node_53"
    },
    "node_54":{
       "status":"Not Started",
       "name":"node_54"
    },
    "node_55":{
       "status":"Not Started",
       "name":"node_55"
    },
    "node_56":{
       "status":"Not Started",
       "name":"node_56"
    },
    "node_57":{
       "status":"Not Started",
       "name":"node_57"
    },
    "node_58":{
       "status":"Not Started",
       "name":"node_58"
    },
    "node_59":{
       "status":"Not Started",
       "name":"node_59"
    },
    "node_60":{
        "status":"Not Started",
        "name":"node_60"
     },
     "node_61":{
        "status":"Not Started",
        "name":"node_61"
     },
     "node_62":{
       "status":"Not Started",
       "name":"node_62"
    },
    "node_63":{
       "status":"Not Started",
       "name":"node_63"
    },
    "node_64":{
       "status":"Not Started",
       "name":"node_64"
    },
    "node_65":{
       "status":"Not Started",
       "name":"node_65"
    },
    "node_66":{
       "status":"Not Started",
       "name":"node_66"
    },
    "node_67":{
       "status":"Not Started",
       "name":"node_67"
    },
    "node_68":{
       "status":"Not Started",
       "name":"node_68"
    },
    "node_69":{
       "status":"Not Started",
       "name":"node_69"
    },
    "node_70":{
        "status":"Not Started",
        "name":"node_70"
     },
     "node_71":{
        "status":"Not Started",
        "name":"node_71"
     },
     "node_72":{
       "status":"Not Started",
       "name":"node_72"
    },
    "node_73":{
       "status":"Not Started",
       "name":"node_73"
    },
    "node_74":{
       "status":"Not Started",
       "name":"node_74"
    },
    "node_75":{
       "status":"Not Started",
       "name":"node_75"
    },
    "node_76":{
       "status":"Not Started",
       "name":"node_76"
    },
    "node_77":{
       "status":"Not Started",
       "name":"node_77"
    },
    "node_78":{
       "status":"Not Started",
       "name":"node_78"
    },
    "node_79":{
       "status":"Not Started",
       "name":"node_79"
    },
    "node_80":{
        "status":"Not Started",
        "name":"node_80"
     },
     "node_81":{
        "status":"Not Started",
        "name":"node_81"
     },
     "node_82":{
       "status":"Not Started",
       "name":"node_82"
    },
    "node_83":{
       "status":"Not Started",
       "name":"node_83"
    },
    "node_84":{
       "status":"Not Started",
       "name":"node_84"
    },
    "node_85":{
       "status":"Not Started",
       "name":"node_85"
    },
    "node_86":{
       "status":"Not Started",
       "name":"node_86"
    },
    "node_87":{
       "status":"Not Started",
       "name":"node_87"
    },
    "node_88":{
       "status":"Not Started",
       "name":"node_88"
    },
    "node_89":{
       "status":"Not Started",
       "name":"node_89"
    }
   },
   "positions":{
      "node_1":[
         500,
         500
      ],
      "node_2":[
        10,
        50
     ],
     "node_3":[
        40,
        50
     ],
     "node_4":[
        70,
        50
     ],
     "node_5":[
        100,
        50
     ],
     "node_6":[
        130,
        50
     ],
     "node_7":[
        160,
        50
     ],
     "node_8":[
        190,
        50
     ],
     "node_9":[
        220,
        50
     ],
     "node_10":[
        250,
        50
     ],
     "node_11":[
        280,
        50
     ],
     "node_12":[
        310,
        50
     ],
     "node_13":[
        340,
        50
     ],
     "node_14":[
        370,
        50
     ],
     "node_15":[
        400,
        50
     ],
     "node_16":[
        430,
        50
     ],
     "node_17":[
        460,
        50
     ],
     "node_18":[
        490,
        50
     ],
     "node_19":[
        520,
        50
     ],
     "node_20":[
        550,
        50
     ],
     "node_21":[
        580,
        50
     ],
     "node_22":[
        610,
        50
     ],
     "node_23":[
        640,
        50
     ],
     "node_24":[
        670,
        50
     ],
     "node_25":[
        700,
        50
     ],
     "node_26":[
        730,
        50
     ],
     "node_27":[
        760,
        50
     ],
     "node_28":[
        790,
        50
     ],
     "node_29":[
        820,
        50
     ],
     "node_30":[
        850,
        50
     ],
     "node_31":[
        880,
        50
     ],
     "node_32":[
        910,
        50
     ],
     "node_33":[
        940,
        50
     ],
     "node_34":[
        970,
        50
     ],
     "node_35":[
        1000,
        50
     ],
     "node_36":[
        1030,
        50
     ],
     "node_37":[
        1060,
        50
     ],
     "node_38":[
        1090,
        50
     ],
     "node_39":[
        1120,
        50
     ],
     "node_40":[
        1150,
        50
     ],
     "node_41":[
        1180,
        50
     ],
     "node_42":[
        1210,
        50
     ],
     "node_43":[
        1240,
        50
     ],
     "node_44":[
        1270,
        50
     ],
     "node_45":[
        1300,
        50
     ],
     "node_46":[
        1330,
        50
     ],
     "node_47":[
        1360,
        50
     ],
     "node_48":[
        1390,
        50
     ],
     "node_49":[
        1420,
        50
     ],
     "node_50":[
        1450,
        50
     ],
     "node_51":[
        1480,
        50
     ],
     "node_52":[
        1510,
        50
     ],
     "node_53":[
        1540,
        50
     ],
     "node_54":[
        1570,
        50
     ],
     "node_55":[
        1600,
        50
     ],
     "node_56":[
        1630,
        50
     ],
     "node_57":[
        1660,
        50
     ],
     "node_58":[
        1690,
        50
     ],
     "node_59":[
        1720,
        50
     ],
     "node_60":[
        1750,
        50
     ],
     "node_61":[
        1780,
        50
     ],
     "node_62":[
        1810,
        50
     ],
     "node_63":[
        1840,
        50
     ],
     "node_64":[
        1870,
        50
     ],
     "node_65":[
        1900,
        50
     ],
     "node_66":[
        1930,
        50
     ],
     "node_67":[
        1960,
        50
     ],
     "node_68":[
        1990,
        50
     ],
     "node_69":[
        2020,
        50
     ],
     "node_70":[
        2050,
        50
     ],
     "node_71":[
        2080,
        50
     ],
     "node_72":[
        2110,
        50
     ],
     "node_73":[
        2140,
        50
     ],
     "node_74":[
        2170,
        50
     ],
     "node_75":[
        2200,
        50
     ],
     "node_76":[
        2230,
        50
     ],
     "node_77":[
        2260,
        50
     ],
     "node_78":[
        2290,
        50
     ],
     "node_79":[
        2320,
        50
     ],
     "node_80":[
        2350,
        50
     ],
     "node_81":[
        2380,
        50
     ],
     "node_82":[
        2410,
        50
     ],
     "node_83":[
        2440,
        50
     ],
     "node_84":[
        2470,
        50
     ],
     "node_85":[
        2500,
        50
     ],
     "node_86":[
        2530,
        50
     ],
     "node_87":[
        2560,
        50
     ],
     "node_88":[
        2590,
        50
     ],
     "node_89":[
        2620,
        50
     ]
   }
}"""

tiledb.plot.widget.Visualize(data=mock_data)

Visualize(value='{\n   "nodes":[\n    "node_1",\n    "node_2",\n    "node_4",\n    "node_6",\n    "node_3",\n …

In [3]:
mock_data = '{"nodes":["node_1","node_2","node_4","node_6","node_3","node_5","node_8","node_7","node_9", "node_10", "node_11"],"edges":[["node_10","node_11"],["node_1","node_2"],["node_1","node_4"],["node_1","node_6"],["node_2","node_3"],["node_4","node_7"],["node_4","node_9"],["node_3","node_5"],["node_3","node_8"]],"node_details":{"node_1":{"status":"Completed","name":"node_1"},"node_2":{"status":"Completed","name":"node_2"},"node_3":{"status":"Failed","name":"node_3"},"node_4":{"status":"Running","name":"node_4"},"node_5":{"status":"Running","name":"node_5"},"node_6":{"status":"Not Started","name":"node_6"},"node_7":{"status":"Cancelled","name":"node_7"},"node_8":{"status":"Not Started","name":"node_8"},"node_9":{"status":"Not Started","name":"node_9"}, "node_10":{"status":"Running","name":"node_10"}, "node_11":{"status":"Not Started","name":"node_11"}}, "positions": { "node_11": [32, 32], "node_10": [42, 230], "node_9": [78, 132], "node_7": [92, 39], "node_6": [178, 232], "node_5": [322, 430], "node_4": [150, 142], "node_3": [12, 230], "node_8": [152, 230], "node_2": [78, 132], "node_1": [34, 319] }}'

tiledb.plot.widget.Visualize(data=mock_data)

Visualize(value='{"nodes":["node_1","node_2","node_4","node_6","node_3","node_5","node_8","node_7","node_9", "…

In [4]:
import tiledb.cloud
import numpy
import random

d = tiledb.cloud.dag.DAG()

# Few base functions
node_1 = d.submit_local(numpy.median, [1, 2, 3], name="node_1")
node_2 = d.submit_local(lambda x: x * 2, node_1, name="node_2")
node_3 = d.submit_local(lambda x: x * 2, node_2, name="node_3")
node_4 = d.submit_local(lambda x: x * 2, node_2, name="node_4")

#Function which sleeps for some time so we can see the graph in different states
def f(x):
    import time
    import random
    if x is None:
        x = random.randrange(0, 30)
    time.sleep(x)
    return x

# Randomly add 96 other nodes to the graph. All of these will use the sleep function
for i in range(4, 100):
    dep = random.randrange(1, i-1)
    # Randomly set dependency on one other node
    node_dep = d.nodes_by_name["node_{}".format(dep)]
    d.submit_local(f, node_dep, name=("node_{}".format(i)))
# print(d._visualize_plotly())
d.compute()
d.visualize()

Visualize(value='{"nodes": ["6465662b-4c09-4e38-9a5e-1437d796dfd6", "c886280b-59e0-4db9-b044-e877f8dcd1a3", "6…

In [6]:
# Import base packages
import tiledb
import tiledb.cloud
from tiledb.cloud.compute import Delayed, DelayedSQL, DelayedArrayUDF
import numpy

x = Delayed(numpy.median)

x

In [7]:
# Build several delayed objects to build in a graph
local = Delayed(lambda x: x * 2, local=True)(100)
array_apply = DelayedArrayUDF("tiledb://TileDB-Inc/quickstart_sparse", lambda x: numpy.sum(x["a"]), name="array_apply")([(1, 4), (1, 4)])
sql = DelayedSQL("select SUM(`a`) as a from `{}`".format("tiledb://TileDB-Inc/quickstart_dense"), name="sql")

# Custom function to use to average all the results we are passing in
def mean(local, array_apply, sql):
    import numpy
    return numpy.mean([local, array_apply, sql.iloc(0)[0]])

res = Delayed(func_exec=mean, name="node_exec")(local, array_apply, sql)

In [9]:
res.visualize(force_plotly=False)

Visualize(value='{"nodes": ["8291eba4-06c3-441e-85e0-16fe287fe7fe", "683f9a11-9c5b-4ed9-84d5-c6afb7a63cde", "0…

In [11]:
# res.compute()

In [13]:
# A few base functions:
import random
node_1 = Delayed(numpy.median, local=True, name="node_1")([1, 2, 3])
node_2 = Delayed(lambda x: x * 2, local=True, name="node_2")(node_1)
node_3 = Delayed(lambda x: x * 2, local=True, name="node_3")(node_2)

nodes_by_name= {'node_1': node_1, 'node_2': node_2, 'node_3': node_3}
#Function which sleeps for some time so we can see the graph in different states
def f():
    import time
    import random
    time.sleep(random.randrange(0, 30))
    return x

# Randomly add 96 other nodes to the graph. All of these will use the sleep function
for i in range(4, 100):
    name = "node_{}".format(i)
    node = Delayed(f, local=True, name=name)()
    
    dep = random.randrange(1, i-1)
    # Randomly set dependency on one other node
    node_dep = nodes_by_name["node_{}".format(dep)]
    # Force the dependency to be set
    node.depends_on(node_dep)
    
    nodes_by_name[name] = node


node_1.visualize()

Visualize(value='{"nodes": ["75257cf7-a467-4d8c-9a63-06400da3840f", "0eab31fa-c20e-4aaa-8d15-b1a04f579c39", "f…